In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.core import *

import torchtext
from torchtext import vocab, data

from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *
from fastai.model import fit
from fastai.dataset import *

## Language modeling

In [3]:
PATH='data/wikitext-2/'
%ls {PATH}

models/  tmp/  wiki.test.tokens  wiki.train.tokens  wiki.valid.tokens


In [4]:
!head -5 {PATH}wiki.train.tokens

 
 = Valkyria Chronicles III = 
 
 Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . <unk> the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more <unk> for series newcomers . Char

In [5]:
!wc -lwc {PATH}wiki.train.tokens

   36718  2051910 10797148 data/wikitext-2/wiki.train.tokens


In [6]:
!wc -lwc {PATH}wiki.valid.tokens

   3760  213886 1121681 data/wikitext-2/wiki.valid.tokens


In [20]:
TEXT = data.Field(lower=True)
FILES = dict(train='wiki.train.tokens', validation='wiki.valid.tokens', test='wiki.test.tokens')
bs = 80
bptt = 70
md = LanguageModelData(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(372, 12981, 1, 2088628)

In [14]:
TEXT.vocab.itos[3875]

'valkyria'

In [8]:
md.trn_ds[0].text[:12]

['<eos>',
 '=',
 'valkyria',
 'chronicles',
 'iii',
 '=',
 '<eos>',
 '<eos>',
 'senjō',
 'no',
 'valkyria',
 '3']

In [27]:
TEXT.vocab.itos[98]

'series'

In [21]:
next(md.trn_dl)

(Variable containing:
      9    409     35  ...      66   1129      2
     11      7   1538  ...     112     14   1919
   3875      2    267  ...       4   2761   5896
         ...            ⋱           ...         
      7      9    509  ...      24     98   2258
      2     11     23  ...       0      5     24
   3875     11  12472  ...      21    414   3479
 [torch.cuda.LongTensor of size 72x80 (GPU 0)], Variable containing:
     11
      7
   1538
   ⋮   
     28
   3668
      4
 [torch.cuda.LongTensor of size 5760 (GPU 0)])

In [22]:
em_sz = 200
nh = 500
nl = 3
learner = md.get_model(SGD_Momentum(0.9), bs, em_sz, nh, nl)

In [23]:
learner.fit(5, 1, wds=1e-6, stepper_fn=reg_rnn_stepper(), clip=0.25)

A Jupyter Widget

[ 0.      5.7723  5.4642]                                   



In [16]:
learner.fit(5, 6, wds=1e-6, stepper_fn=stepper, cycle_len=1, cycle_mult=2, clip=0.25)

A Jupyter Widget

[ 0.      6.2454  5.9691]                                   
[ 1.      5.6808  5.4286]                                   
[ 2.      5.531   5.3216]                                   
[ 3.      5.3978  5.1816]                                   
[ 4.      5.2441  5.0277]                                   
[ 5.      5.1536  4.9435]                                   
[ 6.      5.1131  4.9238]                                   
[ 7.      5.1041  4.8866]                                   
[ 8.      4.9937  4.7999]                                   
[ 9.      4.908   4.7297]                                   
[ 10.       4.8373   4.6767]                                
[ 11.       4.7907   4.6283]                                
[ 12.       4.7495   4.6018]                                
[ 13.       4.7308   4.5833]                                
[ 14.       4.7215   4.5836]                                
[ 15.       4.7991   4.6308]                                
[ 16.       4.7501   4.5

In [19]:
learner.save('lm_423')

In [17]:
math.exp(4.23)

68.7172321738465

### End